In [45]:
import pandas as pd
import os
import numpy as np

In [46]:
window_size = 300 # the size of output frames

In [55]:
output_label = 'train' #'val' 
datafolder = f'{output_label} frames'
filenames = os.listdir(datafolder)

In [49]:
activity_mapping = {
    'S': 0,
    'A': 1,
    'P': 2,
    'T': 3,
    'M': 4,
    'kiné': 5,
    'none': 6  # Handling None values as a separate class
}

In [56]:
X = []
y=[]

for file in filenames:
    path = os.path.join(datafolder,file)

    data_df = pd.read_csv(path)

    windows = [data_df.iloc[i*window_size:(i+1)*window_size] for i in range(len(data_df)// window_size )] #slicing the data

    for windowed_data in windows:

        x = windowed_data[['left_x','left_y','left_z','right_x','right_y','right_z']].to_numpy()

        X.append(x)

        # Map the activities to numbers
        windowed_data['activity_num'] = windowed_data['Activity'].map(activity_mapping)

        # Initialize an empty one-hot encoded matrix
        num_activities = len(activity_mapping)
        one_hot_matrix = np.zeros((len(windowed_data), num_activities), dtype=int)

        # Populate the one-hot encoded matrix
        for idx, value in enumerate(windowed_data['activity_num']):
            one_hot_matrix[idx, value] = 1

        # Create a new DataFrame for the one-hot encoded columns
        columns = [f"activity_{key}" for key in activity_mapping.keys()]
        one_hot_df = pd.DataFrame(one_hot_matrix, columns=columns)

        y.append(one_hot_df.to_numpy())

/tmp/ipykernel_3619/3377331076.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  windowed_data['activity_num'] = windowed_data['Activity'].map(activity_mapping)
/tmp/ipykernel_3619/3377331076.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  windowed_data['activity_num'] = windowed_data['Activity'].map(activity_mapping)
/tmp/ipykernel_3619/3377331076.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [57]:
X_result = np.array(X)
y_result = np.array(y)

print(X_result.shape)
print(y_result.shape)

(225, 300, 6)
(225, 300, 7)


In [58]:
np.save(f'X_{output_label}.npy',X_result)
np.save(f'y_{output_label}.npy',y_result)